In [6]:
import numpy as np
import sklearn
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
import pandas as pd

In [7]:
var_names = ["demand", "trans", "storage", "inertia", "solar"]
regression_out = pd.DataFrame(index=pd.RangeIndex(stop=((len(var_names)**2-len(var_names)))//2), columns=["x", "y", "bx", "by", "bxy", "a", "R^2"])

In [8]:
# Create model 
model = Pipeline([("poly", PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)),
                  ("regression", LinearRegression())])

In [9]:
row = 0

for i in range(len(var_names)):
    for j in range(i+1, len(var_names)):
        # Get data from Scaled Data folder
        raw_data = np.loadtxt("../Data/Normalized Data/" + var_names[i] + "-" + var_names[j] + ".csv",delimiter=",",skiprows=1)
        regression_out.loc[row,["x","y"]] = [var_names[i], var_names[j]] # Set first two columns of DataFrame to names of variables

        # Process and perform regression on data
        x_data = raw_data[:,:-1]
        y_data = raw_data[:,-1].T
        model = model.fit(x_data, y_data)

        # Store regression output in regression_out DataFrame
        regression_out.loc[row, ["bx", "by", "bxy"]] = model["regression"].coef_
        regression_out.loc[row, "a"] = model["regression"].intercept_
        regression_out.loc[row, "R^2"] = model.score(x_data, y_data)

        row += 1

regression_out

,x,y,bx,by,bxy,a,R^2
0,demand,trans,-1.957281,0.679595,-9.340901,0.210968,0.946422
1,demand,storage,-2.166828,-1.34159,50.521534,0.208434,0.799404
2,demand,inertia,-1.908856,-0.052164,1.651915,0.203879,0.738288
3,demand,solar,-2.426346,-1.422451,52.45162,0.213773,0.880668
4,trans,storage,0.775969,-1.326959,-14.561935,0.202406,0.972201
5,trans,inertia,0.828415,-0.049131,-0.747713,0.193451,0.930992
6,trans,solar,0.679419,-1.437876,-4.673853,0.2084,0.961667
7,storage,inertia,-1.011596,-0.045119,1.045347,0.195133,0.704626
8,storage,solar,-1.222305,-1.181673,28.12789,0.202681,0.805526
9,inertia,solar,-0.048894,-1.083337,1.118417,0.198746,0.744912


In [10]:
regression_out.to_csv("regression-output.csv", index=False)